In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures



import pandas as pd

cities_df = pd.read_csv('wojewodztwa_miasta.csv',sep=',')
cities_df = cities_df[(cities_df['nazwa województwa'] == 'PODKARPACKIE') | (cities_df['nazwa województwa'] == 'MAŁOPOLSKIE') | (cities_df['nazwa województwa'] == 'LUBELSKIE')| (cities_df['nazwa województwa'] == 'PODLASKIE')]

In [27]:
from unidecode import unidecode

def remove_polish_chars(text):
    return unidecode(text)

# Zastosuj funkcję do kolumny 'nazwa miasta'
cities_df['nazwa miasta'] = cities_df['nazwa miasta'].apply(remove_polish_chars).str.lower()

cities = cities_df['nazwa miasta']

In [28]:
from bs4 import BeautifulSoup

def max_page_number(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    pagination_links = soup.find_all('a', href=True)  # znajdź wszystkie linki z atrybutem href

    page_numbers = []
    for link in pagination_links:
        page_number = link.text.strip()  # wyciągnij tekst linku i usuń białe znaki z początku i końca
        if page_number.isdigit():  # sprawdź, czy tekst linku zawiera tylko cyfry
            page_numbers.append(int(page_number))  # dodaj numer podstrony do listy
            
    if page_numbers:
        return max(page_numbers)
    else:
        return 1

In [29]:
number = {}
i = 0
for city in cities:
    url = f'https://gratka.pl/nieruchomosci/mieszkania/{city}'
    response = requests.get(url, timeout=2)
    max_page = max_page_number(response)
    number[city] = max_page
    print(i,':',city,':',max_page)
    i+=1



0 : bochnia : 1
1 : nowy wisnicz : 1
2 : brzesko : 1
3 : czchow : 1
4 : alwernia : 1
5 : chrzanow : 5
6 : libiaz : 1
7 : trzebinia : 1
8 : dabrowa tarnowska : 1
9 : szczucin : 1
10 : gorlice : 1
11 : biecz : 1
12 : bobowa : 1
13 : krzeszowice : 1
14 : skala : 1
15 : skawina : 2


KeyboardInterrupt: 

In [30]:
df = pd.DataFrame(list(number.items()), columns=['miasto', 'numer_podstrony'])

In [31]:
df.to_csv('liczba_podstron_z_miastem.csv')
df['miasto'] = df['miasto'].str.replace(' ', '-')
df

,miasto,numer_podstrony
0,bochnia,1
1,nowy-wisnicz,1
2,brzesko,1
3,czchow,1
4,alwernia,1
5,chrzanow,5
6,libiaz,1
7,trzebinia,1
8,dabrowa-tarnowska,1
9,szczucin,1


In [32]:
i=0
data_href_list = []
def gratka(response):
    soup = BeautifulSoup(response.content, 'html.parser')

# Znajdź wszystkie znaczniki article na stronie
    articles = soup.find_all('article', class_='teaserUnified')
    
# Iteruj przez znalezione artykuły
    for article in articles:
        data_href = article.get('data-href', '')    
        print(i+1,data_href)  
        data_href_list.append(data_href)
        


In [33]:
url = f'https://gratka.pl/nieruchomosci/mieszkania/ciechocinek?page=1'
response = requests.get(url, timeout=10)
print(gratka(response))

1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/33905393
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/32265755
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/32002533
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/31723671
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/31354899
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek/ob/26966613
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-ciechocinek-ul-topolowa/ob/34154219
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-ciechocinek-ul-topolowa/ob/34148393
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek-ul-profesora-lorentowicza/ob/34142287
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek-ul-profesora-lorentowicza/ob/34142285
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek-ul-profesora-lorentowicza/ob/34142283
1 https://gratka.pl/nieruchomosci/mieszkanie-ciechocinek-ul-profesora-lorentowicza/ob/34038141
1 https://gratka.pl/nieruchomosci/

In [34]:
for i in range(len(df)):     
  for liczba in range(df['numer_podstrony'][i]):
      # Zakres stron do przeszukania
    url = f'https://gratka.pl/nieruchomosci/mieszkania/{df['miasto'][i]}?page={liczba}'
    response = requests.get(url, timeout=10)
    print(gratka(response))

1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia-ul-legionow-polskich/ob/33314521
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-bochnia-ul-gen-h-dabrowskiego/ob/32260811
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia/ob/34105589
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia/ob/34105591
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia-ul-gen-waleriana-czumy/ob/33003385
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia/ob/34241611
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-bochnia-ul-wygoda/ob/34171331
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-bochnia-ul-krzeczkow/ob/33752137
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia-ul-gen-waleriana-czumy/ob/33752141
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-bochnia-ul-krzeczkow/ob/33752139
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia/ob/33738511
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia-ul-storynka/ob/33325475
1 https://gratka.pl/nieruchomosci/mieszkanie-bochnia-ul-rzeznic

In [45]:
pd.DataFrame(data_href_list).to_csv('linki1.csv')

In [46]:
len(data_href_list)

436

In [37]:
linki = pd.read_csv('linki1.csv')

In [38]:
linki

,Unnamed: 0,0
0,0,https://gratka.pl/nieruchomosci/mieszkanie-cie...
1,1,https://gratka.pl/nieruchomosci/mieszkanie-cie...
2,2,https://gratka.pl/nieruchomosci/mieszkanie-cie...
3,3,https://gratka.pl/nieruchomosci/mieszkanie-cie...
4,4,https://gratka.pl/nieruchomosci/mieszkanie-cie...
...,...,...
431,431,https://gratka.pl/nieruchomosci/nowe-mieszkani...
432,432,https://gratka.pl/nieruchomosci/nowe-mieszkani...
433,433,https://gratka.pl/nieruchomosci/nowe-mieszkani...
434,434,https://gratka.pl/nieruchomosci/nowe-mieszkani...


In [39]:
from multiprocessing.pool import ThreadPool
import pandas as pd
from bs4 import BeautifulSoup

def gratka1(response):
    soup = BeautifulSoup(response.content, 'html.parser')

    data_dict = {}

    try:
        data_dict['region'] = soup.select_one('.offerLocation__region').text.strip()
    except AttributeError:
        data_dict['region'] = ''

    try:
        data_dict['title'] = soup.select_one('.sticker__title').text.strip()
    except AttributeError:
        data_dict['title'] = ''

    try:
        data_dict['total_price'] = soup.select_one('.priceInfo__value').text.strip()
    except AttributeError:
        data_dict['total_price'] = ''

    try:
        data_dict['additional_price_per_m2'] = soup.select_one('.priceInfo__additional').text.strip()
    except AttributeError:
        data_dict['additional_price_per_m2'] = ''

    try:
        opis = soup.find('div', class_='description__rolled ql-container').text.strip()
    except AttributeError:
        opis = ''
    data_dict['opis'] = opis

    li_elements = soup.select('.parameters__singleParameters li')
    for li in li_elements:
        span = li.find('span')
        value = li.find('b', class_='parameters__value')

        if span and value:
            span_text = span.text.strip()
            value_text = value.text.strip()
            data_dict[span_text] = value_text

    li_elements1 = soup.select('.parameters__groupedParameters li')
    for li1 in li_elements1:
        liw = li1.get('data-cy')
        lit = li1.text.strip()
        data_dict[liw] = lit

    img_elements = soup.select('div.gallery__container img')
    src_values = [img['src'] for img in img_elements]
    combined_src = ' '.join(src_values)
    data_dict['links_pictures'] = combined_src

    return pd.DataFrame(data_dict, index=[0])

def process_responses(responses):
    with ThreadPool() as pool:
        results = pool.map(gratka1, responses)

    return pd.concat(results, ignore_index=True)




In [40]:
import pandas as pd
import requests

df1 = pd.DataFrame()

i = 0
for idx in range(0, len(linki['0'])):  
    url = linki['0'][idx]
    print(i)
    i += 1
    response = requests.get(url)
    if response.status_code == 200:
        a = gratka1(response)  # Zakładam, że ta funkcja zwraca DataFrame
        
        # Dodaj kolumnę z URL do DataFrame a
        a['URL'] = url  # Dodaje kolumnę 'URL' z aktualnym URL
        
        # Połącz wynik z ramką danych w każdej iteracji
        df1 = pd.concat([df1, a], ignore_index=True)
    else:
        print(f"Nieudane żądanie dla URL: {url}")

# Wyświetl zaktualizowaną ramkę danych
print(df1)

0


1
2
3
4
         region                                 title  \
0  Ciechocinek,                Mieszkanie Ciechocinek   
1      Brzesko,  Mieszkanie Brzesko, ul. Dworcowa 1/1   
2     Chrzanów,                   Mieszkanie Chrzanów   
3      Gorlice,                    Mieszkanie Gorlice   
4      Skawina,               Nowe mieszkanie Skawina   

                   total_price additional_price_per_m2  \
0  411 000\n      \n        zł          6 737,70 zł/m2   
1  211 000\n      \n        zł          1 749,44 zł/m2   
2  405 000\n      \n        zł          5 686,60 zł/m2   
3  208 000\n      \n        zł          2 773,33 zł/m2   
4  751 591\n      \n        zł          8 492,55 zł/m2   

                                                opis  \
0  Mapa Nieruchomości prezentuje na sprzedaż mies...   
1  PKP S.A. Oddział Gospodarowania Nieruchomościa...   
2  |  Polecam do sprzedaży duże przestronne czter...   
3  Na sprzedaż mieszkanie o powierzchni 75 m2, zl...   
4  Przestrzenny 3 po

In [41]:
df1

,region,title,total_price,additional_price_per_m2,opis,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,Forma własności,...,Numer referencyjny,Możliwy kontakt w języku,balkon,piwnica,links_pictures,URL,Rok budowy,osobne WC,Udogodnienia,taras
0,"Ciechocinek,",Mieszkanie Ciechocinek,411 000\n \n zł,"6 737,70 zł/m2",Mapa Nieruchomości prezentuje na sprzedaż mies...,"Ciechocinek, ...",61 m2,3,1,własność,...,gratka-96/13609/OMS,angielskim,balkon,piwnica,https://d-gr.cdngr.pl/kadry/k/r/gr-ogl/e0/65/3...,https://gratka.pl/nieruchomosci/mieszkanie-cie...,NaN,NaN,NaN,NaN
1,"Brzesko,","Mieszkanie Brzesko, ul. Dworcowa 1/1",211 000\n \n zł,"1 749,44 zł/m2",PKP S.A. Oddział Gospodarowania Nieruchomościa...,"Brzesko, ...","120,61 m2",3,parter,własność,...,gratka-20748471,NaN,NaN,piwnica,https://d-gr.cdngr.pl/kadry/k/r/gr-ogl/3d/e5/2...,https://gratka.pl/nieruchomosci/mieszkanie-brz...,1877,osobne WC,NaN,NaN
2,"Chrzanów,",Mieszkanie Chrzanów,405 000\n \n zł,"5 686,60 zł/m2",| Polecam do sprzedaży duże przestronne czter...,"Chrzanów, ...","71,22 m2",4,4,NaN,...,gratka-MNK-MS-29157,NaN,balkon,NaN,https://d-gr.cdngr.pl/kadry/k/r/gr-ogl/08/81/3...,https://gratka.pl/nieruchomosci/mieszkanie-chr...,1975,NaN,NaN,NaN
3,"Gorlice,",Mieszkanie Gorlice,208 000\n \n zł,"2 773,33 zł/m2","Na sprzedaż mieszkanie o powierzchni 75 m2, zl...","Gorlice, ...",75 m2,2,1,własność,...,gratka-30046,NaN,NaN,piwnica,https://d-gr.cdngr.pl/kadry/k/r/gr-ogl/31/fa/3...,https://gratka.pl/nieruchomosci/mieszkanie-gor...,NaN,NaN,NaN,NaN
4,"Skawina,",Nowe mieszkanie Skawina,751 591\n \n zł,"8 492,55 zł/m2",Przestrzenny 3 pokojowy apartament w luksusowe...,"Skawina, ...","88,50 m2",3,3,własność,...,gratka-1897,NaN,NaN,NaN,https://d-gr.cdngr.pl/kadry/k/r/gr-ogl/0b/07/3...,https://gratka.pl/nieruchomosci/nowe-mieszkani...,2024,NaN,"winda, domofon",taras


In [42]:
df1.to_csv('mieszkanka1648.csv')

In [43]:
# Dzielimy każdy adres po słowie "ul."
df1['Część_po_ul'] = df1['title'].str.split('ul.').str.get(1).str.strip()

# Wyświetlamy ramkę danych z nową kolumną
df1['Część_po_ul']

0             NaN
1    Dworcowa 1/1
2             NaN
3             NaN
4             NaN
Name: Część_po_ul, dtype: object

In [44]:
df1['Lokalizacja']

0    Ciechocinek,                                  ...
1    Brzesko,                                      ...
2    Chrzanów,                                     ...
3    Gorlice,                                      ...
4    Skawina,                                      ...
Name: Lokalizacja, dtype: object